In [1]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2,org.apache.spark:spark-streaming-kafka_2.10:1.6.3,mysql:mysql-connector-java:5.1.17 pyspark-shell'  

In [2]:
import findspark
findspark.init()
import pyspark
import json
import pymysql.cursors
import pymysql.connections
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
sc = SparkContext(appName="datastream")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,15)
#brokers = "52.44.255.113:2181"

In [3]:
def user(con, data):
    with con.cursor() as cur:
        address = data['address']
        channelId = str(address['channelId'])
        user = address['user']
        if channelId == "webchat" or channelId == "directline" or channelId == "test":
            firstName = ""
            lastName = ""
            userId = str("1")
        else:
            userName = user['name'].split(" ")
            print("123",userName)
            firstName = str(userName[0])
            lastName = str(userName[1])
            userId = str(user['id'])
        firstActiveTs = str(data['timestamp'])
        profilePicLink = str(data['profile_pic_link'])
        locale = str(data['locale'])
        timezone = str(data['timezone'])
        gender = str(data['gender'])
        isPaymentEnabled = str(data['is_payment_enabled'])
        markedSpam = str(data['marked_spam'])
        hasBlocked = str(data['has_blocked'])
        print("testingrajgopal", userId)
        sql = "SELECT `user_id` from `user` WHERE `user_id`= " + userId
        cur.execute(sql)
        data = cur.fetchall()
        if len(data) <= 0 :
            sql1 = "INSERT INTO `user` (`user_id`, `first_active_ts`, `first_name`, `last_name`, `profile_pic_link`, `locale`, `timezone`, `gender`, `is_payment_enabled`, `marked_spam`, `has_blocked`) VALUES ('" + userId + "' , '" + firstActiveTs+ "' , '" + firstName+ "' , '" + lastName+ " ',' " + profilePicLink+ " ', '" + locale+ " ', '" + timezone+ " ', '" + gender+ " ', '" + isPaymentEnabled+ " ', '" + markedSpam+ " ', '" + hasBlocked+ "')"
            print("1")
            print(sql1)
            cur.execute(sql1)
    con.commit()

In [4]:
def chatlog(con, data):
    with con.cursor() as cur:
        address = data['address']
        user = address['user']
        bot = address['bot']
        print("botname", str(bot['name']))
        userId = str(user['id'])
        timestamp = str(data['timestamp'])
        address = data['address']
        channelId = str(address['channelId'])
        sessionNumber = str(data['session_id'])
        messageNumber = str(data['message_number'])
        messageType = str(data['message_type'])
        messageChat = con.escape_string( data['message_chat'] )
        print("testing for message_chat", messageChat)
        messageTypeFlag = str(data['message_type_flag'])
        visualSearch = con.escape_string( str(data['visual_search']) )
        if data['product_response_list'] == "":
            productResponseList = json.dumps(str({}))
        else:
            productResponseList = con.escape_string( json.dumps(str(data['product_response_list'])) )
        print("testng for product response list-",productResponseList,"-sss")
        nlpQueryResponse = con.escape_string( str(data['nlp_query_response']) )
        visionFileLink = str(data['vision_file_link'])
        visionEngineResponse = con.escape_string( str(data['vision_engine_response']) )
        quickReplyButton = con.escape_string( str(data['quick_reply_buttons']) )
        sql2 = "INSERT INTO `chatlog` (`user_id`, `timestamp`, `channel_id`, `session_id`, `message_number`, `message_type`, `message_chat`, `message_type_flag`, `visual_search`, `product_response_list`, `nlp_query_response`, `vision_file_link`, `vision_engine_response`, `quick_reply_button` ) VALUES ('" + userId + "' , '" + timestamp + "' , '" + channelId + "' , '" + sessionNumber + "' , '" + messageNumber + "' , '" + messageType + "' , '" + messageChat + "' , '" + messageTypeFlag + "' , '" + visualSearch + "' , '" + str(productResponseList) + "' , '" + nlpQueryResponse + "' , '" + visionFileLink + "' , '" + visionEngineResponse + "' , '" + quickReplyButton + "')"
        print("2")
        print(sql2)
        cur.execute(sql2)
    con.commit()

In [5]:
def click(con, data):
    with con.cursor() as cur:
        address = data['address']
        user = address['user']
        userId = str(user['id'])
        sessionNumber = str(data['session_id'])
        messageNumber = str(data['message_number'])
        timestamp = str(data['timestamp'])
        if data['click_item_sku'] != "" :
            clickItem = str(data['click_item_sku'])
            sql3 = "INSERT INTO `click` (`user_id`, `session_id`, `message_number`, `timestamp`, `click_item`) VALUES ('" + userId + "' , '" + sessionNumber + "' , '" + messageNumber + "' , '" + timestamp + "' , '" + clickItem + "')"
            print("3")
            print(sql3)
            cur.execute(sql3)
    con.commit()    

In [6]:
num = 1000
def takeAndPrint(rdd):
    con = pymysql.connect(host='54.82.9.134', port=3307, user='root', password='', db='metricdb', charset='utf8')
    taken = rdd.take(num + 1)
    for record in taken[:num]:
        row = json.loads(record[1])
        print(row)
        user(con, row)
        chatlog(con, row)
        click(con, row)
    con.close()
topics = {'ashleyexp': 1}
#kafkastream = KafkaUtils.createStream(ssc, "34.203.102.251:2181", 'test-consumer-group', topics)
kafkastream = KafkaUtils.createStream(ssc, "34.203.102.251:2181", 'test-consumer-group', topics)
parsed = kafkastream.foreachRDD(takeAndPrint)

In [7]:
ssc.start()
ssc.awaitTermination()

{u'click_item_sku': u'', u'nlp_query_response': {u'subcategory': [], u'features': [], u'gender': [], u'brand': [], u'colorsavailable': [u'black'], u'xc_category': [u'shirt'], u'entity': u'shirt', u'details': [], u'price': [], u'size': []}, u'locale': u'en_GB', u'text': u'show me black shirt', u'product_response_list': u'', u'agent': u'botbuilder', u'timezone': 5.5, u'visual_search': u'', u'vision_engine_response': u'', u'source': u'facebook', u'type': u'message', u'message_number': 0, u'is_payment_enabled': u'', u'message_chat': u'show me black shirt', u'timestamp': u'2017-05-31T16:48:02.743Z', u'quick_reply_buttons': u'', u'marked_spam': u'', u'address': {u'useAuth': True, u'channelId': u'facebook', u'bot': {u'id': u'1810940785897042', u'name': u'BotCouture'}, u'conversation': {u'id': u'1733841526629600-1810940785897042', u'isGroup': False}, u'user': {u'id': u'1733841526629600', u'name': u'Raj Gopal'}, u'serviceUrl': u'https://facebook.botframework.com', u'id': u'mid.$cAAY1_PnWIhtikK1

{u'click_item_sku': u'', u'nlp_query_response': u'', u'locale': u'', u'text': u'', u'product_response_list': [{u'sku': u'9791754', u'suggested': u'false', u'client_name': u'express', u'xc_sku': u'exp_9791754_c0030', u'subcategory': u'["portofino shirts"]', u'gender': u'women', u'image': u'https://images.express.com/is/image/expressfashion/97_979_1754_058_f74?cache=on&wid=480&fmt=jpeg&qlt=85,1&resmode=sharp2&op_usm=1,1,5,0&defaultImage=Photo-Coming-Soon', u'brand': u'express', u'confidence': u'22.283709', u'productname': u'slim fit convertible sleeve portofino shirt', u'product_url': u'http://www.express.com/clothing/women/slim-fit-convertible-sleeve-portofino-shirt/pro/9791754/cat2490031', u'client_id': u'exp', u'_score': u'22.283709', u'price': u'50.0', u'colorsavailable': [u'pitch black'], u'size': [u'large', u'medium', u'small', u'x large', u'x small', u'xx large', u'xx small', u'xxx large', u'xxxs']}, {u'sku': u'8644260', u'suggested': u'false', u'client_name': u'express', u'xc_sku

('botname', 'BotCouture')
('testing for message_chat', u'')
('testng for product response list-', '\\"[{u\\\'sku\\\': u\\\'9791754\\\', u\\\'suggested\\\': u\\\'false\\\', u\\\'client_name\\\': u\\\'express\\\', u\\\'xc_sku\\\': u\\\'exp_9791754_c0030\\\', u\\\'subcategory\\\': u\\\'[\\\\\\"portofino shirts\\\\\\"]\\\', u\\\'gender\\\': u\\\'women\\\', u\\\'image\\\': u\\\'https://images.express.com/is/image/expressfashion/97_979_1754_058_f74?cache=on&wid=480&fmt=jpeg&qlt=85,1&resmode=sharp2&op_usm=1,1,5,0&defaultImage=Photo-Coming-Soon\\\', u\\\'brand\\\': u\\\'express\\\', u\\\'confidence\\\': u\\\'22.283709\\\', u\\\'productname\\\': u\\\'slim fit convertible sleeve portofino shirt\\\', u\\\'product_url\\\': u\\\'http://www.express.com/clothing/women/slim-fit-convertible-sleeve-portofino-shirt/pro/9791754/cat2490031\\\', u\\\'client_id\\\': u\\\'exp\\\', u\\\'_score\\\': u\\\'22.283709\\\', u\\\'price\\\': u\\\'50.0\\\', u\\\'colorsavailable\\\': [u\\\'pitch black\\\'], u\\\'size\\\

{u'click_item_sku': u'', u'nlp_query_response': u'', u'locale': u'', u'text': u'You can choose to add filters to refine your search results.', u'product_response_list': u'', u'agent': u'botbuilder', u'timezone': u'', u'visual_search': u'', u'vision_engine_response': u'', u'attachments': [{u'content': {u'buttons': [{u'type': u'imBack', u'value': u'Show More', u'title': u'Show More'}, {u'type': u'imBack', u'value': u'Add Filters', u'title': u'Add Filters'}, {u'type': u'imBack', u'value': u'Edit Filters', u'title': u'Edit Filters'}, {u'type': u'imBack', u'value': u'Back To Shopping', u'title': u'Back To Shopping'}, {u'type': u'imBack', u'value': u'New Query', u'title': u'New Query'}, {u'type': u'imBack', u'value': u'Back to Search', u'title': u'Back to Search'}, {u'type': u'imBack', u'value': u'Start Over', u'title': u'Start Over'}, {u'type': u'imBack', u'value': u'Help', u'title': u'Help'}]}, u'contentType': u'application/vnd.microsoft.keyboard'}], u'source': u'facebook', u'type': u'mess

('botname', 'JeanieExp')
('testing for message_chat', u'')
('testng for product response list-', '\\"[{u\\\'sku\\\': u\\\'p-201914898\\\', u\\\'suggested\\\': u\\\'false\\\', u\\\'client_name\\\': u\\\'shoppersstop\\\', u\\\'xc_sku\\\': u\\\'ss_p-201915428_black\\\', u\\\'subcategory\\\': u\\\'[\\\\\\"sports shoes\\\\\\"]\\\', u\\\'gender\\\': u\\\'men\\\', u\\\'image\\\': u\\\'https://sslimages.shoppersstop.com/sys-master/images/h0b/hf7/9912738021406/201915428_9212.jpg_1000Wx1500H\\\', u\\\'brand\\\': u\\\'adidas\\\', u\\\'confidence\\\': u\\\'26.548952\\\', u\\\'productname\\\': u\\\'mens mesh lace up sports shoes\\\', u\\\'product_url\\\': u\\\'https://www.shoppersstop.com/ADIDAS-Mens-Mesh-Lace-Up-Sports-Shoes/p-201915428\\\', u\\\'client_id\\\': u\\\'ss\\\', u\\\'_score\\\': u\\\'26.548952\\\', u\\\'price\\\': u\\\'4699.0\\\', u\\\'colorsavailable\\\': [u\\\'black\\\'], u\\\'size\\\': [u\\\'10\\\', u\\\'7\\\', u\\\'8\\\', u\\\'9\\\']}, {u\\\'sku\\\': u\\\'p-200856475\\\', u\\\'sugg

...

('botname', 'AashishDevBot')
('testing for message_chat', u'')
('testng for product response list-', '\\"[{u\\\'sku\\\': u\\\'p-201272628\\\', u\\\'suggested\\\': u\\\'false\\\', u\\\'client_name\\\': u\\\'shoppersstop\\\', u\\\'xc_sku\\\': u\\\'ss_p-201273157_blue\\\', u\\\'subcategory\\\': u\\\'[\\\\\\"shirts_casual wear\\\\\\"]\\\', u\\\'gender\\\': u\\\'men\\\', u\\\'image\\\': u\\\'https://sslimages.shoppersstop.com/sys-master/images/h1f/hfa/9568447856670/201273157_9308.jpg_1000Wx1500H\\\', u\\\'brand\\\': u\\\'van heusen sport\\\', u\\\'confidence\\\': u\\\'15.580083\\\', u\\\'productname\\\': u\\\'mens regular collar check shirt\\\', u\\\'product_url\\\': u\\\'https://www.shoppersstop.com/VAN-HEUSEN-SPORT-Mens-Regular-Collar-Check-Shirt/p-201273157\\\', u\\\'client_id\\\': u\\\'ss\\\', u\\\'_score\\\': u\\\'15.580083\\\', u\\\'price\\\': u\\\'1999.0\\\', u\\\'colorsavailable\\\': [u\\\'blue\\\'], u\\\'size\\\': [u\\\'39\\\', u\\\'40\\\', u\\\'42\\\', u\\\'44\\\']}, {u\\\'sku\\\'

{u'click_item_sku': u'', u'nlp_query_response': u'', u'locale': u'', u'text': u'You can choose to add filters to refine your search results.', u'product_response_list': u'', u'agent': u'botbuilder', u'timezone': u'', u'visual_search': u'', u'vision_engine_response': u'', u'attachments': [{u'content': {u'buttons': [{u'type': u'imBack', u'value': u'Show More', u'title': u'Show More'}, {u'type': u'imBack', u'value': u'Add Filters', u'title': u'Add Filters'}, {u'type': u'imBack', u'value': u'Back To Shopping', u'title': u'Back To Shopping'}, {u'type': u'imBack', u'value': u'Chat with Specialist', u'title': u'Chat with Specialist'}, {u'type': u'imBack', u'value': u'New Query', u'title': u'New Query'}, {u'type': u'imBack', u'value': u'Back to Search', u'title': u'Back to Search'}, {u'type': u'imBack', u'value': u'Start Over', u'title': u'Start Over'}, {u'type': u'imBack', u'value': u'Help', u'title': u'Help'}]}, u'contentType': u'application/vnd.microsoft.keyboard'}], u'source': u'facebook',

('botname', 'AashishDevBot')
('testing for message_chat', u"Our user Aashish Mishra has shown interest in chatting to you, we are awaiting user\\'s confirmation.")
('testng for product response list-', '"{}"', '-sss')
2
INSERT INTO `chatlog` (`user_id`, `timestamp`, `channel_id`, `session_id`, `message_number`, `message_type`, `message_chat`, `message_type_flag`, `visual_search`, `product_response_list`, `nlp_query_response`, `vision_file_link`, `vision_engine_response`, `quick_reply_button` ) VALUES ('1313298325359965' , '2017-06-01T02:01:02.843Z' , 'facebook' , '8368c907-724e-4abd-a56e-249b2d0da062' , '9' , 'OB' , 'Our user Aashish Mishra has shown interest in chatting to you, we are awaiting user\'s confirmation.' , 'OT' , '' , '"{}"' , '' , '' , '' , '')
{u'click_item_sku': u'', u'nlp_query_response': u'', u'locale': u'en_GB', u'text': u'Yes', u'product_response_list': u'', u'agent': u'botbuilder', u'timezone': 5.5, u'visual_search': u'', u'vision_engine_response': u'', u'source': 

{u'click_item_sku': u'', u'nlp_query_response': u'', u'locale': u'', u'text': u'hi', u'product_response_list': u'', u'agent': u'botbuilder', u'timezone': u'', u'visual_search': u'', u'vision_engine_response': u'', u'source': u'facebook', u'type': u'message', u'message_number': 11, u'is_payment_enabled': u'', u'message_chat': u'hi', u'timestamp': u'2017-06-01T02:01:35.870Z', u'quick_reply_buttons': u'', u'marked_spam': u'', u'address': {u'useAuth': True, u'channelId': u'facebook', u'bot': {u'id': u'672153249630080', u'name': u'AashishDevBot'}, u'conversation': {u'id': u'1313298325359965-672153249630080', u'isGroup': False}, u'user': {u'id': u'1313298325359965', u'name': u'Jesse Samuel'}, u'serviceUrl': u'https://facebook.botframework.com', u'id': u'mid.$cAAI5p5HuWIpikqYLDlcYWDCCPel3'}, u'message_type_flag': u'OT', u'vision_file_link': u'', u'gender': u'', u'has_blocked': u'', u'session_id': u'8368c907-724e-4abd-a56e-249b2d0da062', u'profile_pic_link': u'', u'channel_id': u'facebook', u'

('botname', 'ExpConsumerApp')
('testing for message_chat', u"I\\'m sorry, but I don\\'t understand what you just said. Would you mind rephrasing?")
('testng for product response list-', '"{}"', '-sss')
2
INSERT INTO `chatlog` (`user_id`, `timestamp`, `channel_id`, `session_id`, `message_number`, `message_type`, `message_chat`, `message_type_flag`, `visual_search`, `product_response_list`, `nlp_query_response`, `vision_file_link`, `vision_engine_response`, `quick_reply_button` ) VALUES ('1295200363875776' , '2017-06-01T02:12:15.693Z' , 'facebook' , '11837edf-92b2-4504-a344-074a788358ed' , '1' , 'OB' , 'I\'m sorry, but I don\'t understand what you just said. Would you mind rephrasing?' , 'OT' , '' , '"{}"' , '' , '' , '' , '')
{u'click_item_sku': u'', u'nlp_query_response': u'', u'locale': u'', u'text': u'Please help me improve your shopping experience. Care to share your feedback? It will only take a few seconds!', u'product_response_list': u'', u'agent': u'botbuilder', u'timezone': u''